In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import seaborn as sns
import FinancialMachineLearning as fml

### Chapter 7. Cross-Validation in Finance

#### Exercise 1

금융에 있어서 K-Fold Cross Validation을 수행하기 전에 데이터셋을 섞는 것이 일반적으로 좋지 않은 이유는 무엇인가?데이터를 Shuffling하는 목적은 무엇인가? 데이터를 Shuffling하면 금융 데이터셋에 있어 K-Fold Cross Validation의 목적이 무의미해지는 이유는 무엇인가?

답 : 우리가 사용하는 금융 시계열 데이터셋인데, 만약 데이터셋을 섞게 된다면 순차적 시간 정보다 뒤섞이게 된다. 이는 오히려 추정기의 성능을 떨어뜨리는 결과를 낳게 된다. 일반적으로 Data Science 분야에서 교차 검증을 위해 데이터를 섞는 이유는 무작위 표본 추출로 test, validation, train set을 선택하기 위함인데, 금융 시계열 데이터에서는 Shuffling을 한다면 시간 정보가 사라지고, information leak 등의 다양한 문제가 발생하게 되어 결과적으로 목적이 무의미해진다.

#### Exercise 2

관측된 특성과 레이블로 구성된 한 쌍의 행렬$(X,y)$을 하나 구하자. 3장 연습 문제에서 도출한 데이터셋 중 하나를 사용해도 된다.

In [2]:
df = pd.read_csv('sp500feature.csv')
df

,Unnamed: 0,dollar,frac_diff_dollar,tW,w,bin
0,2009-11-04 14:41:43,50.4500,0.529761,0.750000,1.984399,-1.0
1,2009-11-04 15:43:57,49.8800,-0.137134,0.444444,0.077423,1.0
2,2009-11-05 09:46:35,50.2080,0.349295,0.388889,3.523273,1.0
3,2009-11-06 10:39:27,50.5448,0.038635,0.333333,3.626638,0.0
4,2009-11-06 12:37:18,50.9232,0.559177,0.333333,2.644159,0.0
...,...,...,...,...,...,...
3471,2018-09-26 10:07:41,116.5780,0.334817,0.209694,0.813016,0.0
3472,2018-09-26 15:47:29,115.9960,0.220088,0.189087,0.495700,0.0
3473,2018-09-27 10:02:15,116.1100,0.750801,0.176905,0.055908,0.0
3474,2018-09-27 15:39:35,115.9000,0.349971,0.166905,0.621259,0.0


In [3]:
X = df.iloc[:,:-1].values # dollar, frac_diff, tW, w
y = df.iloc[:,-1].values.reshape(-1,1) # bin

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = False, random_state = 42)

**(a)** $(X,y)$에 대해 데이터를 Shuffling하지 않고 Random Forest Classifier의 10 Fold Cross Validation의 성능을 도출해 보라

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y_train.reshape(1,-1))
weights = compute_class_weight(class_weight = 'balanced', classes = classes, y = y_train.reshape(-1,))
#based on book recommendation
rf = RandomForestClassifier(n_estimators = 1000, criterion = "entropy", bootstrap = True,
                                n_jobs=1, random_state=42, class_weight = 'balanced_subsample', oob_score=False)
cv_gen = KFold(n_splits = 10, shuffle = False)
score = fml.cvScore(rf, X, y, sample_weight = weights, scoring = 'neg_log_loss', cv = None, cvGen = cv_gen, pctEmbargo = 0)
print('rf_clf Mean CV score: {0:.6f}\nCV Variance: {1:.6f}'.format(score.mean(), score.var()))

ValueError: sample_weight.shape == (3,), expected (3128,)!

In [11]:
df[df.bin == -1.0]

,Unnamed: 0,dollar,frac_diff_dollar,tW,w,bin
0,2009-11-04 14:41:43,50.4500,0.529761,0.750000,1.984399,-1.0
7,2009-11-11 10:05:43,52.7001,0.592277,0.297222,0.557362,-1.0
9,2009-11-11 12:43:09,52.5260,0.485345,0.237500,0.292546,-1.0
50,2010-01-20 10:54:08,54.4900,-0.010613,0.256667,3.026698,-1.0
51,2010-01-20 14:52:58,54.6478,0.446691,0.277778,1.421287,-1.0
...,...,...,...,...,...,...
3326,2018-04-30 13:01:26,110.6100,0.733846,0.149752,2.022865,-1.0
3397,2018-06-22 09:46:03,112.0600,0.763690,0.142526,0.426862,-1.0
3398,2018-06-22 10:05:28,112.1400,0.790298,0.142460,1.025435,-1.0
3399,2018-06-25 09:45:01,111.2300,0.351443,0.121854,1.266857,-1.0


In [10]:
weights

array([4.18041237, 0.3827277 , 6.75833333])